In [1]:
import polars as pl
import polars_istr

# IBAN

In [2]:
# First str does not have a valid country code. So not an iban. Second and third are valid.
# Last one has invalid checksum
df = pl.DataFrame({
    "iban": ["AA110011123Z5678", "DE44500105175407324931", "AD1200012030200359100100", "MR0000020001010000123456754"]
})
df.head()

iban
str
"""AA110011123Z56…"
"""DE445001051754…"
"""AD120001203020…"
"""MR000002000101…"


In [3]:
df.select(
    pl.col("iban").iban.country_code().alias("country_code"),
    pl.col("iban").iban.check().alias("reason"),
    pl.col("iban").iban.is_valid().alias("is_valid"),
    pl.col("iban").iban.bban().alias("bban"),
    pl.col("iban").iban.bank_id().alias("bank_id"),
    pl.col("iban").iban.branch_id().alias("branch_id"),
) 

country_code,reason,is_valid,bban,bank_id,branch_id
str,str,bool,str,str,str
null,"""Invalid countr…",false,null,null,null
"""DE""","""ok""",true,"""50010517540732…","""50010517""",null
"""AD""","""ok""",true,"""00012030200359…","""0001""","""2030"""
null,"""Invalid checks…",false,null,null,null


In [4]:
df.select(
    pl.col("iban").iban.extract_all().alias("ib")
).unnest("ib")

country_code,check_digits,bban,bank_id,branch_id
str,str,str,str,str
null,null,null,null,null
"""DE""","""44""","""50010517540732…","""50010517""",null
"""AD""","""12""","""00012030200359…","""0001""","""2030"""
null,null,null,null,null


# ISIN

In [5]:
df = pl.DataFrame(
    {
        "isin": [
            "US0378331005", # AAPL
            "US0378331008", # AAPL w/ bad check digit
            "US037833100", # AAPL w/o check digit
            "CA00206RGB20", # Canadian
            "XS1550212416", # Other
            None,
        ]
    }
)
df.head()

isin
str
"""US0378331005"""
"""US0378331008"""
"""US037833100"""
"""CA00206RGB20"""
"""XS1550212416"""


In [6]:
df.select(
    pl.col("isin").isin.country_code().alias("country_code"),
    pl.col("isin").isin.check_digit().alias("check_digit"),
    pl.col("isin").isin.security_id().alias("security_id"),
    pl.col("isin").isin.is_valid().alias("is_valid"),
)

country_code,check_digit,security_id,is_valid
str,str,str,bool
"""US""","""5""","""037833100""",true
null,null,null,false
null,null,null,false
"""CA""","""0""","""00206RGB2""",true
"""XS""","""6""","""155021241""",true
null,null,null,false


# URL

In [7]:
df = pl.DataFrame(
    {
        "url": [
            "https://example.com/data.csv#row=4",
            "google.com", 
            "ww.google.com", 
            "abc123@email.com", 
            "https://127.0.0.1/", 
            "https://test.com/",
            "file:///tmp/foo",
            "https://example.com/products?page=2&sort=desc",
            None,
        ]
    }
)

In [8]:
df.select(
    pl.col("url").url.host().alias("host"),
    pl.col("url").url.domain().alias("domain"),
    pl.col("url").url.fragment().alias("fragment"),
    pl.col("url").url.path().alias("path"),
    pl.col("url").url.query().alias("query"),
    pl.col("url").url.check().alias("check"),
    pl.col("url").url.is_valid().alias("is_valid"),
    pl.col("url").url.is_special().alias("is_special"),
)

host,domain,fragment,path,query,check,is_valid,is_special
str,str,str,str,str,str,bool,bool
"""example.com""","""example.com""","""row=4""","""/data.csv""",null,"""ok""",true,true
null,null,null,null,null,"""relative URL w…",false,null
null,null,null,null,null,"""relative URL w…",false,null
null,null,null,null,null,"""relative URL w…",false,null
"""127.0.0.1""",null,null,"""/""",null,"""ok""",true,true
"""test.com""","""test.com""",null,"""/""",null,"""ok""",true,true
null,null,null,"""/tmp/foo""",null,"""ok""",true,true
"""example.com""","""example.com""",null,"""/products""","""page=2&sort=de…","""ok""",true,true
null,null,null,null,null,null,null,null
